In [889]:
import matplotlib as plt
import pandas as pd
from lxml import etree

In [890]:
class FitnessProperty :
    def __init__(self, node) :
        self.inverse = node.find('.//Inverse').text
        self.statistic_type = node.find('.//StatisticType').text
        self.weight = node.find('.//Weight').text
        
    def to_string(self) :
        return f'Inverse: {self.inverse}, Statistic Type: {self.statistic_type}, Weight: {self.weight}'

class PartialValue :
    def __init__(self, node) :
        self.type = node.find('.//Type').text
        self.raw_value = node.find('.//RawValue').text
        self.max_value = node.find('.//MaxValue').text
        self.normalized_value = node.find('.//NormalizedValue').text

    def to_string(self) :
        return f'Type: {self.type}, Raw Value: {self.raw_value}, Max Value: {self.max_value}, Normalized Value: {self.normalized_value}'

class Fitness :
    def __init__(self, node) :
        self.value = node.find('.//Value').text
        
        self.partial_values = []
        
        partial_values = node.find('.//PartialValues')
        
        for partial_value in partial_values :
                self.partial_values.append(PartialValue(partial_value))

    def to_string(self) :
        header = f'Value: {self.value}'
        
        partial_values_string = '\nPartial Values: [\n'
        
        for partial_value in self.partial_values :
            partial_values_string += '\t' + partial_value.to_string() + '\n'
            
        partial_values_string += ']'
        
        return header + partial_values_string

In [891]:
class BehaviourGene :
    def __init__(self, node) :
        self.behaviour_type = node.find('.//BehaviourType').text

    def to_string(self) :
        return f'Behaviour Gene [Behaviour Type: {self.behaviour_type}]'
       
class WeaponGene :
    def __init__(self, node) :
        self.weapon_type = node.find('.//WeaponType').text
        
    def to_string(self) :
        return f'Weapon Gene [Weapon Type: {self.weapon_type}]'

In [892]:
class Trait :
    def __init__(self, node) :
        self.identifier = node[0].text
        self.amount = node[1].text
        
    def to_string(self) :    
        return f'Identifier: {self.identifier}, Amount: {self.amount}'
        
class TraitsGene :
    def __init__(self, node) :
        self.chosen_trait = node.find('.//ChosenTrait').text
        self.mutation_added_trait = node.find('.//MutationAddedTrait').text
        self.mutation_removed_trait = node.find('.//MutationRemovedTrait').text
        
        self.traits = []
        
        traits = node.find('.//Traits')
        
        for trait in traits :
            self.traits.append(Trait(trait))
    
    def to_string(self) :
        header = f'TraitGene [Chosen Trait: {self.chosen_trait}, Mutation Added Trait: {self.mutation_added_trait}, Mutation Removed Trait: {self.mutation_removed_trait}'

        traits_string = '\n\tTraits: [\n'
        
        for trait in self.traits :
            traits_string += '\t\t' + trait.to_string() + '\n'
        
        traits_string += '\t\t]\n'
        
        return header + traits_string + '\t]'  

In [893]:
class Chromosome :
    def __init__(self, node) :
        self.genes = []

        genes = node.findall('.//Gene')

        for gene in genes :
            if (gene.items()[0][1] == 'BehaviourGene') :
                self.genes.append(BehaviourGene(gene))
            elif (gene.items()[0][1] == 'TraitsGene') :
                self.genes.append(TraitsGene(gene))
            elif (gene.items()[0][1] == 'WeaponGene') :
                self.genes.append(WeaponGene(gene))
        
    def to_string(self) :
        genes_string = 'Genes: [\n'
        
        for gene in self.genes :
            genes_string += '\t' + gene.to_string() + '\n'
        
        genes_string += ']\n'
        
        return genes_string

class Creature :
    def __init__(self, node) :
        self.id = node.find('.//Id').text
        self.generation = node.find('.//Generation').text
        self.fitness = Fitness(node.find('.//Fitness'))
        self.chromosome = Chromosome(node.find('.//Chromosome')) 
        
        self.parents = []

        parents = node.find('.//Parents')
        
        for parent in parents :
            self.parents.append(parent.text)

        self.children = []
        
        children = node.find('.//Children')
        
        for child in children :
            self.children.append(child.text)        
        
    def to_string(self) :
        header = f'Id: {self.id}, Generation: {self.generation}'
        
        parents_string = f'\nParents: {self.parents}'
        
        children_string = f'\nChildren: {self.children}'
        
        fitness_string = '\n' + self.fitness.to_string()
        
        chromosome_string = '\n' + self.chromosome.to_string()
        
        return header + parents_string + children_string + fitness_string + chromosome_string

In [894]:
class Generation :
    def __init__(self, node) :
        self.number = node.find('.//Number').text
        self.generation_fitness = node.find('.//GenerationFitness').text
        
        self.creatures = []
        
        creatures = node.find('.//Creatures')
        
        for creature in creatures :
            self.creatures.append(Creature(creature))
            
    def to_string(self) : 
        header = f'Number: {self.number}, Generation Fitness: {self.generation_fitness}'
        
        creatures_string = '\nCreatures: [\n'
        
        for creature in self.creatures :
            creatures_string += '\t' + creature.to_string() + '\n'
            
        creatures_string += ']'
        
        return header + creatures_string

In [895]:
class GeneticAlgorithmData :
    def __init__(self, node) :
        self.team = node.find('.//Team').text
        self.trait_selection_amount = node.find('.//TraitSelectionAmount').text
        self.trait_selection_dumbness = node.find('.//TraitSelectionDumbness').text
        
        self.fitness_properties = []
        
        fitness_properties = node.find('.//FitnessProperties')
        
        for fitness_property in fitness_properties :
            self.fitness_properties.append(FitnessProperty(fitness_property))
        
        self.generations = []
        
        generations = node.find('.//Generations')
        
        for generation in generations :
            self.generations.append(Generation(generation))
            
    def print_fitness_properties(self) :
        for fitness_property in self.fitness_properties :
            print(fitness_property.to_string())
            
    def to_string(self) :
        header = f'Team: {self.team},'\
                f' Trait Selection Amount: {self.trait_selection_amount},'\
                f' Trait Selection Dumbness: {self.trait_selection_dumbness}'

        fitness_property_string = '\nFitness Properties: [\n'
        
        for fitness_property in self.fitness_properties :
            fitness_property_string += '\t' + fitness_property.to_string() + '\n'
                
        fitness_property_string += ']'
                
        generations_string = '\nGenerations: [\n'
        
        for generation in self.generations :
            generations_string += '\t' + generation.to_string() + '\n'
            
        generations_string += ']'
        
        return header + fitness_property_string + generations_string

In [896]:
team1_file_path = "./team1.xml"

tree = etree.parse(team1_file_path)

root = tree.getroot()

data = GeneticAlgorithmData(root)

print(data.generations[0].creatures[0].to_string())

Id: 1, Generation: 1
Parents: []
Children: []
Value: 0.482088357
Partial Values: [
	Type: PositiveHitsTaken, Raw Value: 0, Max Value: 3, Normalized Value: 0
	Type: PositiveHitsDealt, Raw Value: 0, Max Value: 0, Normalized Value: 0
	Type: NegativeHitsTaken, Raw Value: 0, Max Value: 13, Normalized Value: 0.110461272
	Type: NegativeHitsDealt, Raw Value: 0, Max Value: 67, Normalized Value: 0.110461272
	Type: DamageDealt, Raw Value: 0, Max Value: 14.7999945, Normalized Value: 0
	Type: DamageTaken, Raw Value: 0, Max Value: 2, Normalized Value: 0.110461272
	Type: HealingDealt, Raw Value: 0, Max Value: 0, Normalized Value: 0
	Type: HealingTaken, Raw Value: 0, Max Value: 0.450000018, Normalized Value: 0
	Type: FriendlyFire, Raw Value: 0, Max Value: 0, Normalized Value: 0.150704533
]
Genes: [
	Behaviour Gene [Behaviour Type: Support]
	TraitGene [Chosen Trait: None, Mutation Added Trait: None, Mutation Removed Trait: None
	Traits: [
		Identifier: LessDamageReductionMoreDamage, Amount: 1
		]
	]
	W